# Time Series Analysis of NYC Yellow Taxi Trips

In [ ]:
%pip install kagglehub
%pip install pandas
%pip install matplotlib
%pip install numpy
%pip install seaborn
%pip install statsmodels
%pip install pmdarima

In [ ]:
import kagglehub
import seaborn as sns
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from datetime import datetime
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm
from pmdarima import auto_arima  # Per ottimizzazione automatica dei parametri

## Dataset loading

In [ ]:
path = kagglehub.dataset_download("elemento/nyc-yellow-taxi-trip-data")
file_path1 = os.path.join(path, "yellow_tripdata_2015-01.csv")
file_path2 = os.path.join(path, "yellow_tripdata_2016-01.csv")
file_path3 = os.path.join(path, "yellow_tripdata_2016-02.csv")
file_path4 = os.path.join(path, "yellow_tripdata_2016-03.csv")

file_paths = [
    #file_path1,
    file_path2,
    file_path3,
    file_path4]


Escludiamo il primo file CSV dal momento che esso è relativo al gennaio 2015, mostrando una separazione di numerosi mesi con gli altri file CSV, cosa che porterebbe a dei problemi nella strutturazione e nell'analisi della serie temporale risultante.

In [ ]:
# Columns of interest
columns = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'trip_distance', 'fare_amount', 'total_amount']

# Load the dataset into a pandas dataframe
dfs = [pd.read_csv(f, usecols=columns, parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime']) for f in file_paths]
df = pd.concat(dfs, ignore_index=True)

## Data cleaning

In [ ]:
df = df[(df['trip_distance'] > 0) & (df['fare_amount'] > 0) & (df['total_amount'] > 0)]

In [ ]:
df['pickup_date'] = df['tpep_pickup_datetime'].dt.date
df['trip_duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60
df = df[(df['trip_duration'] > 0) & (df['trip_duration'] < 240)]

Creiamo due nuove colonne utilizzate, una come variabile temporale, ed entrambe per escludere altri record particolari (ulteriore data cleaning)

## Stationarity analysis

In [ ]:
daily_trips = df.groupby('pickup_date').size()

ts_daily_trips = daily_trips.copy()
ts_daily_trips.index = pd.to_datetime(ts_daily_trips.index)
ts_daily_trips = ts_daily_trips.asfreq('D', fill_value=0)

In [ ]:
ts_daily_trips['2016-01-23'] = ts_daily_trips[ts_daily_trips.index.dayofweek == 5].mean()
ts_daily_trips['2016-01-24'] = ts_daily_trips[ts_daily_trips.index.dayofweek == 6].mean()
ts_daily_trips['2016-01-25'] = ts_daily_trips[ts_daily_trips.index.dayofweek == 0].mean()
ts_daily_trips['2016-01-26'] = ts_daily_trips[ts_daily_trips.index.dayofweek == 1].mean()

Questo "riempimento" con le medie è necessario perché nei giorni "riempiti" erano presenti dei valori mancanti, cosa che può causare problemi in una serie temporale. L'approccio di riempimento basato sulla media è stato approvato e va bene così.

In [ ]:
# Train and test
train_size = int(len(ts_daily_trips) * 0.8)
train_ts = ts_daily_trips[:train_size]
test_ts = ts_daily_trips[train_size:]

La serie temporale è stata splittata in una parte di training e una di test, in modo tale da poter valutare le predizioni effettuate dai modelli che si costruiranno sulla base dell'andamento reale.

In [ ]:
plt.figure(figsize=(14, 7))
ts_daily_trips.plot()
plt.title('Daily NYC Taxi Trips')
plt.xlabel('Date')
plt.ylabel('Number of Trips')
plt.grid()
plt.show()

In [ ]:
def test_stationarity_with_interpretation(timeseries):
    result = adfuller(timeseries)
    print('=== Augmented Dickey-Fuller Test ===')
    print(f'ADF Statistic: {result[0]:.4f}')
    print(f'p-value: {result[1]}')
    print('Critical Values:')
    for key, value in result[4].items():
        print(f'   {key}: {value:.4f}')
    
    # Interpretazione del risultato
    if result[1] <= 0.05:
        print("\n✅ La serie è stazionaria")
    else:
        print("\n❌ La serie NON è stazionaria")

# Test sulla serie originale
print("\n[TEST SULLA SERIE ORIGINALE]")
test_stationarity_with_interpretation(train_ts)

The $p$-value is less than $0.05$, which means that we can reject the null hypothesis that the time series is non-stationary. The time series is stationary.

### Automatic optimization of parameters with auto_arima

In [ ]:
auto_arima_model = auto_arima(train_ts, d=0, seasonal=False, trace=True, error_action='ignore', suppress_warnings=True)
print("Best ARIMA model:", auto_arima_model.summary())

In [ ]:
auto_sarima_model = auto_arima(train_ts, d=0, seasonal=True, m=7, trace=True, error_action='ignore', suppress_warnings=True)
print("Best SARIMA model:", auto_sarima_model.summary())

The auto_arima function calls, have shown that the best parameters are:
- 3, 0, 3 for ARIMA
- 1, 0, 1, 2, 1, 1, 7 for SARIMA

However, those results does not reflect the choice of not differentiating the series (also, the results with those parameters were not so good). So, we will choose the parameters using the ACF and PACF plots.

## ARIMA-SARIMA models
### p, d, q parameters

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

plt.figure(figsize=(12, 6))
plt.subplot(121)
plot_acf(train_ts, ax=plt.gca(), lags=20)
plt.subplot(122)
plot_pacf(train_ts, ax=plt.gca(), lags=20)
plt.show()

La struttura dei grafici ACF e PACF, unitamente all’analisi preliminare sulla stazionarietà della serie, suggerisce la presenza di componenti autoregressive e di media mobile di basso ordine. Nel dettaglio, il significativo picco della PACF al primo lag indica che un termine autoregressivo di ordine uno (p=1) è adeguato per catturare la dipendenza immediata della serie dai propri valori passati. Allo stesso tempo, l’ACF che decresce gradualmente verso zero ma non fornisce un taglio netto suggerisce l’opportunità di inserire un piccolo termine MA, portando all’impiego di un modello ARIMA(1,0,1) non stagionale.

Poiché la serie presenta una chiara stagionalità settimanale (un ciclo di 7 giorni), si è deciso di estendere la struttura ARIMA a SARIMA, introducendo una componente stagionale con periodo s=7. L’analisi dei picchi su ACF e PACF ai multipli di 7 lag consente di discernere se inserire un termine AR o MA stagionale. L’individuazione di un significativo spike nella PACF a lag stagionali può guidare verso l’introduzione di un termine AR stagionale (P=1), mentre un picco di autocorrelazione stagionale nell’ACF motiva l’inclusione di un termine MA stagionale (Q=1). Nel caso in esame, l’ipotesi iniziale è di partire da un modello SARIMA(1,0,1)(1,0,0)[7], eventualmente valutando l’aggiunta di un termine MA stagionale se i residui mostrano ancora autocorrelazione sistematica.

In [ ]:
model = ARIMA(train_ts, order=(1, 0, 1))

arima_fit = model.fit()

print("\n=== ARIMA Model Summary ===")
print(arima_fit.summary())

In [ ]:
# Risultato del training:
"""
=== ARIMA Model Summary ===
                               SARIMAX Results
==============================================================================
Dep. Variable:                      y   No. Observations:                   72
Model:                 ARIMA(1, 0, 1)   Log Likelihood                -828.204
Date:                Sun, 08 Dec 2024   AIC                           1664.408
Time:                        11:19:36   BIC                           1673.514
Sample:                    01-01-2016   HQIC                          1668.033
                         - 03-12-2016
Covariance Type:                  opg
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       3.825e+05   7924.222     48.264      0.000    3.67e+05    3.98e+05
ar.L1          0.3719      0.159      2.332      0.020       0.059       0.685
ma.L1          0.6023      0.184      3.270      0.001       0.241       0.963
sigma2      5.748e+08      0.154   3.72e+09      0.000    5.75e+08    5.75e+08
===================================================================================
Ljung-Box (L1) (Q):                   0.10   Jarque-Bera (JB):                 4.94
Prob(Q):                              0.75   Prob(JB):                         0.08
Heteroskedasticity (H):               1.19   Skew:                            -0.60
Prob(H) (two-sided):                  0.68   Kurtosis:                         2.55
===================================================================================
"""

In [ ]:
# Diagnostic plots
arima_fit.plot_diagnostics(figsize=(12,8))
plt.show()

In [ ]:
model_sarima = sm.tsa.statespace.SARIMAX(train_ts, order=(1, 0, 1), seasonal_order=(1, 0, 0, 7))
sarima_fit = model_sarima.fit(method='bfgs')

print(sarima_fit.summary())

In [ ]:
# Risultati del training:
"""
SARIMAX Results
==========================================================================================
Dep. Variable:                                  y   No. Observations:                   72
Model:             SARIMAX(1, 0, 1)x(1, 0, [], 7)   Log Likelihood                -824.441
Date:                            Sun, 08 Dec 2024   AIC                           1656.882
Time:                                    12:26:02   BIC                           1665.989
Sample:                                01-01-2016   HQIC                          1660.508
                                     - 03-12-2016
Covariance Type:                              opg
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.9826      0.043     22.837      0.000       0.898       1.067
ma.L1         -0.3073      0.340     -0.905      0.365      -0.973       0.358
ar.S.L7        0.7655      0.188      4.078      0.000       0.398       1.133
sigma2      8.156e+08   4.74e-11   1.72e+19      0.000    8.16e+08    8.16e+08
===================================================================================
Ljung-Box (L1) (Q):                   0.38   Jarque-Bera (JB):                 0.64
Prob(Q):                              0.54   Prob(JB):                         0.73
Heteroskedasticity (H):               0.64   Skew:                             0.21
Prob(H) (two-sided):                  0.28   Kurtosis:                         2.81
===================================================================================
"""

In [ ]:
sarima_fit.plot_diagnostics(figsize=(12, 8))
plt.show()

In [ ]:
forecast_arima = arima_fit.get_forecast(steps=20)
forecast_sarima = sarima_fit.get_forecast(steps=20)

results = {
    'ARIMA': (forecast_arima.predicted_mean, forecast_arima.se_mean, forecast_arima.conf_int()),
    'SARIMA': (forecast_sarima.predicted_mean, forecast_sarima.se_mean, forecast_sarima.conf_int())
}

arima_fc_series = pd.Series(results['ARIMA'][0], index=test_ts.index)
sarima_fc_series = pd.Series(results['SARIMA'][0], index=test_ts.index)

arima_lower_series = pd.Series(results['ARIMA'][2].iloc[:, 0], index=test_ts.index)
arima_upper_series = pd.Series(results['ARIMA'][2].iloc[:, 1], index=test_ts.index)

sarima_lower_series = pd.Series(results['SARIMA'][2].iloc[:, 0], index=test_ts.index)
sarima_upper_series = pd.Series(results['SARIMA'][2].iloc[:, 1], index=test_ts.index)

plt.figure(figsize=(14, 7), dpi=300)
plt.plot(train_ts, label='Training')
plt.plot(test_ts, label='Test')
plt.plot(arima_fc_series, label='ARIMA Forecast', color='red')
plt.plot(sarima_fc_series, label='SARIMA Forecast', color='green')
plt.ylim(200000, 600000)
plt.fill_between(arima_lower_series.index, arima_lower_series, arima_upper_series, color='red', alpha=0.3)
plt.fill_between(sarima_lower_series.index, sarima_lower_series, sarima_upper_series, color='green', alpha=0.3)
plt.title('Forecast Comparison')
plt.legend(loc='upper left', fontsize=10)
plt.grid()
plt.show()

I risultati mostrano un'ottima predizione da parte del modello SARIMA(1, 0, 1)(1, 0, 0, 7), che si avvicina di molto all'andamento reale del dataset di test, nonostante gli intervalli di confidenza che, via via, aumentano esponenzialmente, non rappresentando un ottimo intervallo. Il modello ARIMA(1, 0, 1), invece, mostra una andamento che, dopo circa 3 step, si assesta su un andamento costante, mostrando un risultato pessimo.

## Hour based analysis

Seconda analisi di serie temporale: dopo aver effettuato la prima analisi sul dataset trasformato in serie temporale ma sulla base di parametri giornalieri, proviamo ad effettuare un'analisi su misurazioni orarie. In questo modo dimostriamo di saper gestire serie temporali di tipologie diverse e valutiamo anche il comportamento dei modelli in condizioni operative differenti.

In [ ]:
# Specificare i file rilevanti per gennaio, febbraio e marzo 2016
hour_dfs = [pd.read_csv(f, usecols=columns, parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime']) 
            for f in file_paths]
hour_df = pd.concat(hour_dfs, ignore_index=True)

In [ ]:
# Rimuovere righe con valori nulli
hour_df.dropna(subset=columns, inplace=True)

# Filtrare viaggi con valori negativi o anomali
hour_df = hour_df[(hour_df['trip_distance'] > 0) & 
                  (hour_df['fare_amount'] > 0) & 
                  (hour_df['total_amount'] > 0)]

# Calcolare la durata del viaggio
hour_df['trip_duration'] = (hour_df['tpep_dropoff_datetime'] - hour_df['tpep_pickup_datetime']).dt.total_seconds() / 60

# Filtrare viaggi con durata negativa o eccessiva (>4 ore)
hour_df = hour_df[(hour_df['trip_duration'] > 0) & (hour_df['trip_duration'] < 240)]

Stessa ETL effettuata precedentemente

In [ ]:
# Creare un indice temporale arrotondando al più vicino inizio dell'ora
hour_df['pickup_hour'] = hour_df['tpep_pickup_datetime'].dt.floor('H')

# Aggregare il numero di viaggi per ogni ora
hourly_trips = hour_df.groupby('pickup_hour').size()

# Trasformare in una serie temporale con frequenza fissa
hourly_ts = hourly_trips.asfreq('H', fill_value=0)

# Fill all the hours of January 23-24-25-26, 2016 with the average of the corresponding hours of the corresponding days-of-week in the rest of the dataset
# NOT the average of the corresponding days-of-week, but the average of the corresponding hours
# Riempimento delle ore mancanti basato su medie dello stesso giorno della settimana
for i in range(24):
    hourly_ts[f'2016-01-23 {i:02d}:00:00'] = hourly_ts[
        (hourly_ts.index.hour == i) & (hourly_ts.index.dayofweek == 5)
    ].mean()
    hourly_ts[f'2016-01-24 {i:02d}:00:00'] = hourly_ts[
        (hourly_ts.index.hour == i) & (hourly_ts.index.dayofweek == 6)
    ].mean()


# Train and test
train_size = int(len(hourly_ts) * 0.8)
train_hourly_ts = hourly_ts[:train_size]
test_hourly_ts = hourly_ts[train_size:]

Stesse operazioni effettuate precedentemente ma su base oraria. Anche il riempimento dei valori mancanti è stato effettuato per gli stessi motivi visti precedentemente

In [ ]:
plt.figure(figsize=(14, 7))
train_hourly_ts.plot()
plt.title('Numero di viaggi per ora (gennaio-febbraio 2016)')
plt.xlabel('Ora')
plt.ylabel('Numero di viaggi')
plt.grid()
plt.show()


In [ ]:
print("\n[TEST SULLA SERIE ORIGINALE]")
test_stationarity_with_interpretation(train_hourly_ts)

La serie è stazionaria: non serve differenziarla

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, pacf

acf_values = acf(train_hourly_ts, nlags=30)
pacf_values = pacf(train_hourly_ts, nlags=30)
print("ACF:", acf_values)
print("PACF:", pacf_values)

Valori "printati" di ACF e PACF, in modo da poter capire ancora meglio rispetto ai grafici sottostanti l'andamento e la scelta dei parametri dei modelli

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(121)
plot_acf(train_hourly_ts, ax=plt.gca(), lags=20)
plt.subplot(122)
plot_pacf(train_hourly_ts, ax=plt.gca(), lags=20)
plt.show()

Osservando i picchi di autocorrelazione a lag ravvicinati e l’evidente stagionalità giornaliera (24 ore), è stato selezionato un modello SARIMA in grado di catturare sia la componente non stagionale che quella stagionale a 24 ore. La presenza di un forte picco all’ACF sul primo lag e la struttura residuale in PACF suggeriscono un termine AR e un termine MA non stagionali di basso ordine (ad esempio ARIMA(1,0,1)). L’inclusione di componenti stagionali AR e MA sul periodo di 24 ore, come SARIMA(1,0,1)(1,0,1,24), consente di modellare efficacemente il pattern ripetitivo giornaliero.

In [ ]:
hourly_ts_log = np.log1p(train_hourly_ts)

hourly_arima_model = ARIMA(hourly_ts_log, order=(1, 0, 1))

hourly_arima_fit = hourly_arima_model.fit()

print("\n=== ARIMA Model Summary ===")
print(hourly_arima_fit.summary())

NOTA: in questo caso abbiamo effettuato una normalizzazione con np.log1p perché i valori erano troppo "traballanti"

In [ ]:
# Risultato del training:
"""
=== ARIMA Model Summary ===
                               SARIMAX Results
==============================================================================
Dep. Variable:                      y   No. Observations:                 1747
Model:                 ARIMA(1, 0, 1)   Log Likelihood              -16135.310
Date:                Sun, 08 Dec 2024   AIC                          32278.621
Time:                        12:27:01   BIC                          32300.484
Sample:                    01-01-2016   HQIC                         32286.703
                         - 03-13-2016
Covariance Type:                  opg
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.587e+04    662.957     23.940      0.000    1.46e+04    1.72e+04
ar.L1          0.8457      0.016     52.695      0.000       0.814       0.877
ma.L1          0.6122      0.008     75.517      0.000       0.596       0.628
sigma2      6.154e+06   1.27e+05     48.528      0.000    5.91e+06     6.4e+06
===================================================================================
Ljung-Box (L1) (Q):                  55.24   Jarque-Bera (JB):             20484.33
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               1.90   Skew:                             0.33
Prob(H) (two-sided):                  0.00   Kurtosis:                        19.76
===================================================================================
"""

In [ ]:
hourly_arima_fit.plot_diagnostics(figsize=(12,8))
plt.show()

In [ ]:
# Applicare la trasformazione logaritmica
hourly_ts_log = np.log1p(train_hourly_ts)

# Modello SARIMA
hourly_model_sarima = sm.tsa.statespace.SARIMAX(hourly_ts_log, order=(1, 0, 1), seasonal_order=(1, 0, 1, 24))
hourly_sarima_fit = hourly_model_sarima.fit()

print(hourly_sarima_fit.summary())

In [ ]:
# Risultato del training:
"""
                                     SARIMAX Results
==========================================================================================
Dep. Variable:                                  y   No. Observations:                 2184
Model:             SARIMAX(1, 0, 1)x(1, 0, 1, 24)   Log Likelihood                -285.741
Date:                            Sun, 08 Dec 2024   AIC                            581.481
Time:                                    12:27:03   BIC                            609.926
Sample:                                01-01-2016   HQIC                           591.879
                                     - 03-31-2016
Covariance Type:                              opg
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.9988      0.001    747.181      0.000       0.996       1.001
ma.L1          0.0548      0.003     16.586      0.000       0.048       0.061
ar.S.L24       0.9995      0.001   1487.106      0.000       0.998       1.001
ma.S.L24      -0.9750      0.015    -64.387      0.000      -1.005      -0.945
sigma2         0.0737      0.000    190.866      0.000       0.073       0.074
===================================================================================
Ljung-Box (L1) (Q):                   0.06   Jarque-Bera (JB):           8518933.29
Prob(Q):                              0.81   Prob(JB):                         0.00
Heteroskedasticity (H):               3.70   Skew:                             2.45
Prob(H) (two-sided):                  0.00   Kurtosis:                       308.93
===================================================================================
"""

In [ ]:
hourly_sarima_fit.plot_diagnostics(figsize=(12, 8))
plt.show()

In [ ]:
hourly_forecast_arima = hourly_arima_fit.get_forecast(steps=len(test_hourly_ts))
hourly_forecast_sarima_log = hourly_sarima_fit.get_forecast(steps=len(test_hourly_ts))
hourly_forecast_sarima = np.expm1(hourly_forecast_sarima_log.predicted_mean)
hourly_forecast_sarima_se = hourly_forecast_sarima_log.se_mean
hourly_forecast_sarima_ci = np.expm1(hourly_forecast_sarima_log.conf_int())

# Allineare gli indici delle previsioni con il test set
arima_fc_series = pd.Series(hourly_forecast_arima.predicted_mean.values, index=test_hourly_ts.index)
sarima_fc_series = pd.Series(hourly_forecast_sarima.values, index=test_hourly_ts.index)

arima_lower_series = pd.Series(hourly_forecast_arima.conf_int().iloc[:, 0].values, index=test_hourly_ts.index)
arima_upper_series = pd.Series(hourly_forecast_arima.conf_int().iloc[:, 1].values, index=test_hourly_ts.index)

sarima_lower_series = pd.Series(hourly_forecast_sarima_ci.iloc[:, 0].values, index=test_hourly_ts.index)
sarima_upper_series = pd.Series(hourly_forecast_sarima_ci.iloc[:, 1].values, index=test_hourly_ts.index)

from sklearn.metrics import mean_squared_error, mean_absolute_error

# Calcolo delle metriche di errore
arima_rmse = np.sqrt(mean_squared_error(test_hourly_ts, arima_fc_series))
sarima_rmse = np.sqrt(mean_squared_error(test_hourly_ts, sarima_fc_series))
arima_mae = mean_absolute_error(test_hourly_ts, arima_fc_series)
sarima_mae = mean_absolute_error(test_hourly_ts, sarima_fc_series)

print(f"ARIMA RMSE: {arima_rmse:.2f}, MAE: {arima_mae:.2f}")
print(f"SARIMA RMSE: {sarima_rmse:.2f}, MAE: {sarima_mae:.2f}")

plt.figure(figsize=(14, 7), dpi=300)
plt.plot(train_hourly_ts, label='Training')
plt.plot(test_hourly_ts, label='Test')
plt.plot(arima_fc_series, label='ARIMA Forecast', color='red')
plt.plot(sarima_fc_series, label='SARIMA Forecast', color='green')
plt.ylim(-1000, 35000)
plt.fill_between(arima_lower_series.index, arima_lower_series, arima_upper_series, color='red', alpha=0.3)
plt.fill_between(sarima_lower_series.index, sarima_lower_series, sarima_upper_series, color='green', alpha=0.3)
plt.title(f'Forecast Comparison (ARIMA RMSE: {arima_rmse:.2f}, SARIMA RMSE: {sarima_rmse:.2f})')
plt.legend(loc='upper left', fontsize=10)
plt.grid()
plt.show()

I risultati della predizione mostrano un andamento costante pessimo per il modello ARIMA(1, 0, 1), mentre un andamento molto migliore per il modello SARIMA(1, 0, 1)(1, 0, 1, 24), che segue la stagionalità. Tuttavia, le predizioni del modello SARIMA, dopo qualche step, sembrano seguire un andamento che, via via, tende ad abbassarsi, mostrando come, a lungo andare, la predizione potrebbe essere pressoché sbagliata. Inoltre, i valori di RMSE risultano essere particolarmente non ottimali, essendo nell'ordine di grandezza delle migliaia (ARIMA RMSE=, SARIMA RMSE=, ARIMA MAE=, SARIMA MAE=).

Per questo motivo, si è deciso di introdurre un'ulteriore stagionalità settimanale, continuando a considerare la stagionalità giornaliera. Per fare ciò, la serie è stata differenziata con parametro 24.

In [ ]:
# Differenziazione per la stagionalità giornaliera (24 ore)
hourly_ts_diff = hourly_ts.diff(24).dropna()

plt.figure(figsize=(12, 6))
plt.subplot(121)
plot_acf(hourly_ts_diff, ax=plt.gca(), lags=20)
plt.subplot(122)
plot_pacf(hourly_ts_diff, ax=plt.gca(), lags=20)
plt.show()

I grafici di autocorrelazione e autocorrelazione parziale seguono un andamento estremamente simile ai grafici visti nel caso precedente, quindi si sceglie un modello SARIMA con gli stessi parametri del precedente, ma con stagionalità di 24*7

In [ ]:
# Suddivisione del dataset in training e test (80/20)
split_ratio = 0.8
split_index = int(len(hourly_ts_diff) * split_ratio)
train_hourly_ts = hourly_ts_diff[:split_index]
test_hourly_ts = hourly_ts_diff[split_index:]

# Modello SARIMA considerando la stagionalità settimanale
sarima_model = sm.tsa.statespace.SARIMAX(train_hourly_ts,
                                         order=(1, 0, 1),  # Parametri ARIMA
                                         seasonal_order=(1, 0, 1, 24 * 7))  # Stagionalità settimanale
sarima_fit = sarima_model.fit()

# Previsioni per il set di test
forecast_steps = len(test_hourly_ts)  # Lunghezza del set di test
hourly_forecast_sarima = sarima_fit.get_forecast(steps=forecast_steps)
forecast_mean = hourly_forecast_sarima.predicted_mean
forecast_ci = hourly_forecast_sarima.conf_int()

# Allineare gli indici delle previsioni con il test set
sarima_fc_series = pd.Series(forecast_mean.values, index=test_hourly_ts.index)
sarima_lower_series = pd.Series(forecast_ci.iloc[:, 0].values, index=test_hourly_ts.index)
sarima_upper_series = pd.Series(forecast_ci.iloc[:, 1].values, index=test_hourly_ts.index)

# Calcolo delle metriche di errore
sarima_rmse = np.sqrt(mean_squared_error(test_hourly_ts, sarima_fc_series))
sarima_mae = mean_absolute_error(test_hourly_ts, sarima_fc_series)

print(f"SARIMA RMSE: {sarima_rmse:.2f}, MAE: {sarima_mae:.2f}")

# Visualizzazione dei risultati
plt.figure(figsize=(14, 7), dpi=300)
plt.plot(train_hourly_ts, label='Training')
plt.plot(test_hourly_ts, label='Test', color='blue')
plt.plot(sarima_fc_series, label='SARIMA Forecast', color='green')
plt.fill_between(sarima_lower_series.index, sarima_lower_series, sarima_upper_series, color='green', alpha=0.3)
plt.title(f'SARIMA Forecast with Weekly Seasonality (RMSE: {sarima_rmse:.2f})')
plt.xlabel('Date')
plt.ylabel('Values')
plt.legend(loc='upper left', fontsize=10)
plt.grid()
plt.show()

In questi risultati, SARIMA si comporta estremamente bene. Osservando, infatti, in corrispondenza del test dataset, si nota come l'andamento predetto da SARIMA segue molto bene l'andamento reale.